# Cleaning Data

This notebook is used in order to clean the metadata retrieved with the software Arcas.

In [1]:
import glob
import pandas as pd

In [2]:
dfs = []
for filename in glob.glob('../data/Auction_*.json'):
    dfs.append(pd.read_json(filename))

In [3]:
df = pd.concat(dfs, ignore_index=True, sort=False)

In [4]:
df.provenance.unique()

array(['IEEE', 'arXiv', 'Nature', 'Springer'], dtype=object)

In [5]:
len(df.title.unique())

3461

In [6]:
len(df.unique_key.unique())

3569

In [7]:
provenance_size = df.groupby(['unique_key', 'provenance']).size().reset_index().groupby('provenance').size()
provenance_size

provenance
IEEE         257
Nature       213
Springer    1301
arXiv       1798
dtype: int64

In [8]:
df.to_json('../data/auction_November_2018.json')

Cleaning
--------

In [9]:
df = pd.read_json('../data/auction_November_2018.json')

In [10]:
# Initial all letter in the string author are lowercased.
df.author = df.author.str.lower()

In [11]:
#from fuzzywuzzy import fuzz
import itertools

In [ ]:
import tqdm

We can output the names that are very similar but the last check has to be manually.

In [ ]:
temp = df

In [ ]:
pairs = itertools.combinations(temp.author.unique(), 2)

In [ ]:
to_check = []
for i, j in tqdm.tqdm(pairs):
    ratio = fuzz.token_set_ratio(i,j)
    if ratio >=90 and ratio != 100:
        to_check.append((i, j))

In [ ]:
to_check

In [ ]:
df[df['author'] == 'r.grossman']['title'].unique()

In [ ]:
df[df['author'] == 'd.coyle']['title'].unique()

Duplicate articles
------------------

In [12]:
table = df.groupby(['title', 'unique_key']).size().reset_index().groupby('title').count()
duplicates = table[table['unique_key']>1]
duplicates

,unique_key,0
title,,
\n,2,2
A Game-Theoretic Approach to Energy Trading in the Smart Grid,2,2
A Grey-Box Approach to Automated Mechanism Design,2,2
A Speculative Futures Market with Zero-Intelligence,2,2
Ad Exchange: Envy-Free Auctions with Mediators,2,2
An Online Multi-unit Auction with Improved Competitive Ratio,2,2
Analyses of Cardinal Auctions,2,2
Auctioneer,2,2
Auctions,11,11


In [13]:
duplicates_title = df[df['title'].isin(duplicates.index)]['title'].unique()

In [14]:
duplicates_in_arxiv = df[(df['title'].isin(duplicates.index)) & (df['provenance'] == 'arXiv')]['title'].unique()

In [15]:
diff = list(set(duplicates_title) - set(duplicates_in_arxiv))

In [16]:
df_without_arxiv = df[~(df['provenance']=='arXiv')]

In [17]:
df_without_arxiv = df_without_arxiv.drop_duplicates(subset='title')

In [18]:
# df_without_arxiv.to_json('../data/pd_November_2018_without_arxiv.json')

**Drop duplicates.**

In [19]:
articles_to_drop = df[(df['title'].isin(duplicates.index)) & (df['provenance']=='arXiv')]['unique_key'].unique()

In [20]:
df = df[~df['unique_key'].isin(articles_to_drop)]

In [21]:
df = df.drop_duplicates(subset='title')

In [22]:
len(df['title'].unique()), len(df['unique_key'].unique())

(3458, 3458)

In [43]:
df = df[~(df['date'] < 1974)]

**Export clean json.**

In [44]:
df.to_json('../data/auction_November_2018_clean.json')